In [1]:
import torch
import os
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm_notebook
from tqdm import tqdm
from PIL import Image
from torchvision import datasets, models, transforms, models, utils
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim import lr_scheduler
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
# import torch_optimizer as optim
import random
#from ipywidgets import IntProgress
import time
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [2]:
# gpu or cpu
if torch.cuda.is_available():
  device = torch.device("cuda")
  # device name, number
  print(device,torch.cuda.get_device_name(0),torch.cuda.device_count())
else:
  device = torch.device("cpu")
  print(device)

cuda Tesla V100-SXM2-32GB 2


# Data Loader

In [3]:
df = pd.read_csv('./training_data_resize/training_data.csv')
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2028 entries, 0 to 2027
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            2028 non-null   object 
 1   light         2028 non-null   object 
 2   ground_truth  2028 non-null   float64
 3   grade         2028 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 63.5+ KB
None
                                        id light  ground_truth  grade
0  00130747_A_3457_20200715_100727_7.5.jpg     A           7.5      1
1  00130747_A_3458_20200715_100736_7.5.jpg     A           7.5      1
2  00130747_A_4810_20200812_112534_7.9.jpg     A           7.9      1
3  00130747_A_4811_20200812_112539_7.9.jpg     A           7.9      1
4  00241567_A_6659_20200916_153156_7.6.jpg     A           7.6      1


In [4]:
# data, label
file_id = df["id"].to_numpy()
light = df["light"].to_numpy()
ground_truth = df['ground_truth'].to_numpy()

file = []
number = []
for i in range(len(light)):
    file.append(os.path.join("./training_data_resize", light[i], file_id[i]))
    number.append(ground_truth[i])


#print(file)
#print(number)
#print(type(file))
#print(type(number))
#print(len(file))
#print(len(number))


# shuffle data
temp = list(zip(file, number))
random.shuffle(temp)
file, number = zip(*temp)

file = np.array(file)
number = np.array(number)

#train validation data 7:3
TVrate = 0.7
file_train = file[:int(len(file)*TVrate)]
file_vali = file[int(len(file)*TVrate):]
print(len(file_train))
#print(len(file_vali))

number_train = number[:int(len(file)*TVrate)]
number_vali = number[int(len(file)*TVrate):]
#print(number_train)

1419


In [5]:
preprocess = transforms.Compose([
    transforms.Scale(256),
    transforms.CenterCrop(210),
    #transforms.Grayscale(num_output_channels=3),
    transforms.RandomAffine(degrees=30),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def default_loader(path):
    img_pil = Image.open(path)
    #img_pil = img_pil.resize((224, 224))
    img_tensor = preprocess(img_pil)
    return img_tensor

class train_set(Dataset):
    def __init__(self, loader=default_loader):
        self.images = file_train
        self.target = number_train
        self.loader = loader

    def __getitem__(self, index):
        fn = self.images[index]
        img = self.loader(fn)
        target = self.target[index]
        return img, target

    def __len__(self):
        return len(self.images)

class valid_set(Dataset):
    def __init__(self, loader=default_loader):
        self.images = file_vali
        self.target = number_vali
        self.loader = loader

    def __getitem__(self, index):
        fn = self.images[index]
        img = self.loader(fn)
        target = self.target[index]
        return img, target

    def __len__(self):
        return len(self.images)

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [6]:
train_data = train_set()
trainloader = DataLoader(train_data, batch_size=64, shuffle=True,pin_memory=True)
valid_data = valid_set()
validloader = DataLoader(valid_data, batch_size=32, shuffle=True,pin_memory=True)
print(len(trainloader))
print(len(validloader))

23
20


# Model

In [7]:
#!pip install efficientnet-pytorch
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=1)

# resuming training
#model.load_state_dict(torch.load(img_path + '/modelb0_2/model'+ str(12)+'.pth'))
#summary(model.to('cuda:0'), (3, 224, 224))


Loaded pretrained weights for efficientnet-b7


In [8]:
'''
#from torchsummary import summary
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(512, 3)
)
#summary(model.to('cuda:0'), (3, 224, 224))
'''

"\n#from torchsummary import summary\nmodel = models.resnet18(pretrained=True)\nmodel.fc = nn.Sequential(\n    nn.Dropout(0.5),\n    nn.Linear(512, 3)\n)\n#summary(model.to('cuda:0'), (3, 224, 224))\n"

In [9]:
'''
from torchsummary import summary
model = models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096,3)

summary(model.to('cuda:0'), (3, 224, 224))
'''

"\nfrom torchsummary import summary\nmodel = models.alexnet(pretrained=True)\nmodel.classifier[6] = nn.Linear(4096,3)\n\nsummary(model.to('cuda:0'), (3, 224, 224))\n"

In [ ]:
model = torch.nn.DataParallel(model, device_ids=None, output_device=None, dim=0).to(device)
criterion = nn.MSELoss(reduction = 'mean').to(device)
#criterion = nn.CrossEntropyLoss().to(device)
#optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
#optimizer = optim.Ranger(params=model.parameters(), lr=0.0001)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.7)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100, eta_min=0, last_epoch=-1)
epoch_start = 1
epochs = 1000

os.mkdir("./b7_r_cos")
for epoch in range(epoch_start, epochs+1):
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    valid_loss_min = 0.0
    train_acc = 0
    val_acc = 0
    start = time.time()
    
    # write loss in txt file
    f = open('./b7_r_cos/model.txt', 'a')
    f.write('running epoch: {}'.format(epoch))
    f.close()
    print('running epoch: {}'.format(epoch))

    ### train the model
    model.train()
    #for data, target in tqdm(trainloader):
    for data, target in trainloader:
        target = target.float()
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        output = torch.squeeze(output)
        loss = criterion(target, output)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)     
        err = torch.abs(output-target)
        train_acc += torch.sum((err<=1.5).float())


    ### validate the model
    model.eval()
    #for data, target in tqdm(validloader):
    for data, target in validloader:
        target = target.float()
        data, target = data.to(device), target.to(device)
        output = model(data)
        output = torch.squeeze(output)
        loss = criterion(target, output)
        valid_loss += loss.item()*data.size(0)       
        err = torch.abs(output-target)         
        val_acc += torch.sum((err<=1.5).float())

    # update lr
    scheduler.step()
    
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(validloader.dataset)

    train_acc = int(train_acc)/int(len(file)*0.7)
    val_acc = int(val_acc)/int(len(file)*0.3)
    end = time.time()
    
    print('\tTraining Loss: {:.6f} \tTraining Acc: {:.4f} \tValidation Loss: {:.6f} \tValidation Acc: {:.4f} \tTime: {:.2f}'.format(
        train_loss, train_acc, valid_loss, val_acc, end-start))
    # write loss in txt file
    f = open('./b7_r_cos/model.txt', 'a')
    f.write('\tTraining Loss: {:.6f} \tTraining Acc: {:.4f} \tValidation Loss: {:.6f} \tValidation Acc: {:.4f} \tTime: {:.2f}\n'.format(
        train_loss, train_acc, valid_loss, val_acc, end-start))
    f.close()
    #writer.add_scalar("Loss/train", train_loss, epoch)
    

    # save model
    if train_acc>0.98 and val_acc>0.93:
        #torch.save(model, './b7_r_cos_3/model'+ str(epoch) +'.pth')

running epoch: 1


/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


	Training Loss: 57.357299 	Training Acc: 0.1078 	Validation Loss: 1894.759663 	Validation Acc: 0.0000 	Time: 29.77
running epoch: 2
	Training Loss: 6.947866 	Training Acc: 0.4426 	Validation Loss: 453.142773 	Validation Acc: 0.0016 	Time: 25.33
running epoch: 3
	Training Loss: 5.158090 	Training Acc: 0.4905 	Validation Loss: 271.989147 	Validation Acc: 0.0000 	Time: 25.09
running epoch: 4
	Training Loss: 4.464994 	Training Acc: 0.5222 	Validation Loss: 17.746018 	Validation Acc: 0.2599 	Time: 24.96
running epoch: 5
	Training Loss: 3.353773 	Training Acc: 0.6054 	Validation Loss: 5.877316 	Validation Acc: 0.4885 	Time: 24.76
running epoch: 6
	Training Loss: 2.645784 	Training Acc: 0.6554 	Validation Loss: 3.743365 	Validation Acc: 0.5740 	Time: 24.75
running epoch: 7
	Training Loss: 2.281200 	Training Acc: 0.6977 	Validation Loss: 3.021274 	Validation Acc: 0.6464 	Time: 24.31
running epoch: 8
	Training Loss: 1.981027 	Training Acc: 0.7181 	Validation Loss: 2.543362 	Validation Acc: 0.68

	Training Loss: 0.285867 	Training Acc: 0.9887 	Validation Loss: 1.279704 	Validation Acc: 0.8651 	Time: 24.47
running epoch: 66
	Training Loss: 0.300283 	Training Acc: 0.9873 	Validation Loss: 1.154833 	Validation Acc: 0.8783 	Time: 24.62
running epoch: 67
	Training Loss: 0.288405 	Training Acc: 0.9915 	Validation Loss: 1.316940 	Validation Acc: 0.8618 	Time: 24.24
running epoch: 68
	Training Loss: 0.260972 	Training Acc: 0.9915 	Validation Loss: 1.098544 	Validation Acc: 0.9095 	Time: 24.29
running epoch: 69
	Training Loss: 0.300299 	Training Acc: 0.9894 	Validation Loss: 1.225057 	Validation Acc: 0.8766 	Time: 24.33
running epoch: 70
	Training Loss: 0.244921 	Training Acc: 0.9937 	Validation Loss: 1.384648 	Validation Acc: 0.8520 	Time: 24.92
running epoch: 71
	Training Loss: 0.230355 	Training Acc: 0.9993 	Validation Loss: 1.127064 	Validation Acc: 0.8816 	Time: 24.71
running epoch: 72
	Training Loss: 0.227409 	Training Acc: 0.9951 	Validation Loss: 1.115191 	Validation Acc: 0.8914

	Training Loss: 0.178679 	Training Acc: 0.9993 	Validation Loss: 1.016637 	Validation Acc: 0.8964 	Time: 24.92
running epoch: 130
	Training Loss: 0.190668 	Training Acc: 0.9958 	Validation Loss: 1.113376 	Validation Acc: 0.8832 	Time: 25.38
running epoch: 131
	Training Loss: 0.186946 	Training Acc: 1.0000 	Validation Loss: 1.131598 	Validation Acc: 0.8964 	Time: 25.03
running epoch: 132
	Training Loss: 0.196651 	Training Acc: 0.9979 	Validation Loss: 0.990780 	Validation Acc: 0.8849 	Time: 25.33
running epoch: 133
	Training Loss: 0.215546 	Training Acc: 0.9944 	Validation Loss: 1.290359 	Validation Acc: 0.8684 	Time: 25.66
running epoch: 134
	Training Loss: 0.225317 	Training Acc: 0.9972 	Validation Loss: 1.250877 	Validation Acc: 0.8799 	Time: 25.40
running epoch: 135
	Training Loss: 0.204521 	Training Acc: 0.9993 	Validation Loss: 1.281721 	Validation Acc: 0.8882 	Time: 24.91
running epoch: 136
	Training Loss: 0.239107 	Training Acc: 0.9986 	Validation Loss: 1.275222 	Validation Acc:

	Training Loss: 0.501291 	Training Acc: 0.9626 	Validation Loss: 2.059126 	Validation Acc: 0.8092 	Time: 24.65
running epoch: 196
	Training Loss: 0.419223 	Training Acc: 0.9760 	Validation Loss: 1.482353 	Validation Acc: 0.8635 	Time: 24.62
running epoch: 197
	Training Loss: 0.321045 	Training Acc: 0.9859 	Validation Loss: 1.220351 	Validation Acc: 0.8865 	Time: 24.81
running epoch: 198
	Training Loss: 0.360505 	Training Acc: 0.9796 	Validation Loss: 1.136131 	Validation Acc: 0.8783 	Time: 24.48
running epoch: 199
	Training Loss: 0.259650 	Training Acc: 0.9922 	Validation Loss: 1.236323 	Validation Acc: 0.8931 	Time: 24.37
running epoch: 200
	Training Loss: 0.275788 	Training Acc: 0.9930 	Validation Loss: 1.003562 	Validation Acc: 0.8947 	Time: 24.55
running epoch: 201
	Training Loss: 0.315197 	Training Acc: 0.9915 	Validation Loss: 1.236419 	Validation Acc: 0.8799 	Time: 24.61
running epoch: 202
	Training Loss: 0.564333 	Training Acc: 0.9605 	Validation Loss: 2.023952 	Validation Acc:

	Training Loss: 0.144839 	Training Acc: 0.9993 	Validation Loss: 0.982159 	Validation Acc: 0.9079 	Time: 25.43
running epoch: 260
	Training Loss: 0.137792 	Training Acc: 0.9986 	Validation Loss: 0.989603 	Validation Acc: 0.9112 	Time: 25.18
running epoch: 261
	Training Loss: 0.157337 	Training Acc: 0.9993 	Validation Loss: 0.969596 	Validation Acc: 0.9112 	Time: 24.91
running epoch: 262
	Training Loss: 0.150186 	Training Acc: 1.0000 	Validation Loss: 1.026167 	Validation Acc: 0.9030 	Time: 25.08
running epoch: 263
	Training Loss: 0.135201 	Training Acc: 1.0000 	Validation Loss: 1.023336 	Validation Acc: 0.9128 	Time: 24.94
running epoch: 264
	Training Loss: 0.131291 	Training Acc: 0.9993 	Validation Loss: 1.023985 	Validation Acc: 0.9095 	Time: 25.12
running epoch: 265
	Training Loss: 0.123373 	Training Acc: 1.0000 	Validation Loss: 1.049098 	Validation Acc: 0.9128 	Time: 25.18
running epoch: 266
	Training Loss: 0.130890 	Training Acc: 0.9986 	Validation Loss: 1.116210 	Validation Acc:

	Training Loss: 0.108440 	Training Acc: 1.0000 	Validation Loss: 0.866053 	Validation Acc: 0.9276 	Time: 25.86
running epoch: 324
	Training Loss: 0.109002 	Training Acc: 1.0000 	Validation Loss: 0.936420 	Validation Acc: 0.9145 	Time: 25.30
running epoch: 325
	Training Loss: 0.111997 	Training Acc: 1.0000 	Validation Loss: 1.005499 	Validation Acc: 0.9161 	Time: 25.20
running epoch: 326
	Training Loss: 0.123349 	Training Acc: 0.9993 	Validation Loss: 0.868346 	Validation Acc: 0.9145 	Time: 25.46
running epoch: 327
	Training Loss: 0.126805 	Training Acc: 0.9972 	Validation Loss: 0.993023 	Validation Acc: 0.9293 	Time: 25.26
running epoch: 328
	Training Loss: 0.153849 	Training Acc: 0.9958 	Validation Loss: 1.412148 	Validation Acc: 0.8882 	Time: 25.67
running epoch: 329
	Training Loss: 0.203744 	Training Acc: 0.9937 	Validation Loss: 1.178991 	Validation Acc: 0.8964 	Time: 25.57
running epoch: 330
	Training Loss: 0.236386 	Training Acc: 0.9937 	Validation Loss: 1.734758 	Validation Acc:

	Training Loss: 0.308232 	Training Acc: 0.9859 	Validation Loss: 1.240488 	Validation Acc: 0.8947 	Time: 24.91
running epoch: 388
	Training Loss: 0.246372 	Training Acc: 0.9951 	Validation Loss: 1.306245 	Validation Acc: 0.8816 	Time: 25.07
running epoch: 389
	Training Loss: 0.377076 	Training Acc: 0.9803 	Validation Loss: 2.027791 	Validation Acc: 0.7714 	Time: 24.92
running epoch: 390
	Training Loss: 0.311861 	Training Acc: 0.9852 	Validation Loss: 1.765509 	Validation Acc: 0.8503 	Time: 24.55
running epoch: 391
	Training Loss: 0.258896 	Training Acc: 0.9930 	Validation Loss: 1.733150 	Validation Acc: 0.8289 	Time: 24.81
running epoch: 392
	Training Loss: 0.395366 	Training Acc: 0.9803 	Validation Loss: 1.380568 	Validation Acc: 0.8766 	Time: 24.77
running epoch: 393
	Training Loss: 0.530238 	Training Acc: 0.9605 	Validation Loss: 2.241527 	Validation Acc: 0.8174 	Time: 25.08
running epoch: 394
	Training Loss: 0.609188 	Training Acc: 0.9507 	Validation Loss: 3.292821 	Validation Acc:

	Training Loss: 0.172914 	Training Acc: 0.9965 	Validation Loss: 1.051569 	Validation Acc: 0.8964 	Time: 25.26
running epoch: 452
	Training Loss: 0.187771 	Training Acc: 0.9972 	Validation Loss: 1.421059 	Validation Acc: 0.8832 	Time: 24.69
running epoch: 453
	Training Loss: 0.174421 	Training Acc: 0.9972 	Validation Loss: 0.912992 	Validation Acc: 0.9145 	Time: 24.91
running epoch: 454
	Training Loss: 0.173011 	Training Acc: 0.9958 	Validation Loss: 1.089179 	Validation Acc: 0.9062 	Time: 24.73
running epoch: 455
	Training Loss: 0.145163 	Training Acc: 0.9979 	Validation Loss: 0.945500 	Validation Acc: 0.9095 	Time: 24.78
running epoch: 456
